In [14]:
import sys
input_fn = sys.argv[1]
#info_fn = sys.argv[2]
#input_fn = "S1-0Gy-1.combination.Mutect_Strelka_union.vcf.readinfo.filtered.vcf"
info_fn = "N1-S1.combination.Mutect_Strelka_union.vcf.readinfo"
output_fn = input_fn.replace(".vcf",".net_clonal.vcf")
input_file = file(input_fn)
info_file = file(info_fn)
output_file = file(output_fn,'w')
output1_file = file(output_fn.replace(".vcf",".excluded.vcf"),'w')

input_line = input_file.readline()
output_file.write(input_line)
input_line = input_file.readline()
info_line = info_file.readline()
info_line = info_file.readline()

while input_line:
    new=0
    input_split = input_line.split('\t')
    #print input_split[0].replace("X",20).replace("Y",21).replace("MT",22)
    input_chr = int(input_split[0].replace("X",'20').replace("Y",'21').replace("MT",'22'))
    input_pos = long(input_split[1])
    
    while info_line:
        info_split = info_line.split('\t')
        info_chr = int(info_split[0].replace("X",'20').replace("Y",'21').replace("MT",'22'))
        info_pos = long(info_split[1])
        
        if input_chr < info_chr:
            new=1
            break
        elif input_chr >info_chr:
            info_line = info_file.readline()
            continue
        else:
            if input_pos < info_pos:
                new=1
                break
            elif input_pos > info_pos:
                info_line = info_file.readline()
                continue
            else:
                new=0  
                break
                
    if new == 1:
        output_file.write(input_line)
    else:
        output1_file.write(input_line)
        
    input_line = input_file.readline()
        


In [2]:
import sys, pysam
input_fn = sys.argv[1]
#input_fn = "S1-0Gy-1.combination.Mutect_Strelka_union.vcf.readinfo.filtered"
output_fn = input_fn + "N1S1_read_removal.vcf"
input_file = file(input_fn)
output_file = file(output_fn,'w')
#output1_file = file((output_fn+'N1S1_excluded.vcf'),'w')
bam_file = pysam.AlignmentFile('/home/users/jhyouk/06_mm10_SNUH_radiation/03_bam/N1-S1.s.md.ir.br.rg.bam','rb')

input_line = input_file.readline().strip()
output_file.write(input_line + '\t'+ 'N1S1_var_read_N' +'\n')
#output1_file.write(input_line+ '\t'+ 'N1S1_var_read_N' +'\n')
input_line = input_file.readline().strip()

while input_line:
    tf=0
    input_split = input_line.split('\t')
    input_chr = input_split[0]
    input_pos = long(input_split[1])
    alt_nt = input_split[4][0]
    
    for read1 in bam_file.pileup(input_chr,input_pos-1,input_pos):
        if long(read1.pos)==input_pos-1:
            for pileupread in read1.pileups:
                if pileupread.query_position == None:
                    continue
                elif pileupread.alignment.query_sequence[pileupread.query_position] == alt_nt:
                    tf+=1
                else:
                    'blank'
    
    if tf<2:
        output_file.write(input_line+'\t'+str(tf)+'\n')
    
    input_line = input_file.readline().strip()

In [32]:
import math
import scipy.stats

temp = scipy.stats.binom(30,0.2)
temp.pmf()


0.0